In [1]:
import pandas as pd
import numpy as np
import datetime as dt

Lectura inicial

In [119]:
eventsData = pd.read_csv("data/Eventos(+completo).csv",low_memory = False)


Feature con la cantidad de static page por usuario

In [120]:
staticPageData = pd.DataFrame({'cantidadDeStaticPage' :eventsData.groupby(['person','staticpage']).size()})
staticPageData.reset_index(inplace = True)
staticPageData = staticPageData.groupby('person').agg({'cantidadDeStaticPage':'sum'}).reset_index()


Feature con la cantidad de veces que entra a un url de compra

In [121]:
#No funciono
auxiliar = pd.DataFrame({'cantidad_de_Url_de_compra' :eventsData.groupby(['person','url']).size()})
auxiliar = auxiliar.reset_index()
auxiliar["url"] = auxiliar['url'].map(lambda x: x.split('/')[1])
auxiliar = auxiliar.groupby('person').agg({'cantidad_de_Url_de_compra':'sum'}).reset_index()


Feature con la cantidad de busquedas que realiza un usuario por termino

In [122]:
#No aporta mucho
searchData = pd.DataFrame({'Cantidad_De_Busquedas_Por_Termino' :eventsData.groupby(['search_term','person','event']).size()})
searchData = searchData.sort_values(by ='Cantidad_De_Busquedas_Por_Termino',ascending= False )
searchData.reset_index(inplace = True)
searchData.set_index('person',inplace = True)
searchData = searchData.drop('event',axis=1)
searchData.reset_index(inplace=True)


Prototipos

In [123]:
#No aporta mucho
conversionData = eventsData[eventsData['event'].map(lambda x: ( x== 'conversion'))]
conversionData.set_index('person', inplace = True)
conversionData = conversionData.loc[:,['model','condition']]


In [124]:
#No sirvio
dataF = pd.DataFrame({'cantidad_de_Url_de_compra' :eventsData.groupby(['person','skus']).size()})
dataF = dataF.reset_index()
dataF['sku_multiple']= dataF['skus'].map(lambda x: len(x.split(',')))
dataF = dataF.groupby('person').agg({'sku_multiple':'sum'}).reset_index()

Feature que usa searchData y conversionData. Analiza con la cantidad de busquedas por termino y la cantidad de veces que compro lo que busco

In [125]:
#Aumento en 2%
dataSearchAndSelling = pd.merge(searchData,conversionData.reset_index(),on="person",how="inner")
dataSearchAndSelling
dataSearchAndSelling["model"] = dataSearchAndSelling["model"].fillna("Desconocido")
dataSearchAndSelling["marca"] = dataSearchAndSelling["model"].map(lambda x:"apple" if(x.split(" ")[0] in ["iphone","ipad"]) else x.split(" ")[0])
dataSearchAndSelling["search_term"] = dataSearchAndSelling['search_term'].map(lambda x: 'samsung' if x.split(" ")[0].lower() in ['s6','s8','s7','samsung'] else x.split(' ')[0])
dataSearchAndSelling["search_term"] = dataSearchAndSelling['search_term'].map(lambda x: 'motorola' if x.split(" ")[0].lower() in ['motorola','moto'] else x.split(' ')[0])
dataSearchAndSelling["search_term"] = dataSearchAndSelling['search_term'].map(lambda x: 'iphone' if x.split(" ")[0].lower() in ['iphone'] else x.split(' ')[0])
dataSearchAndSelling['search_and_purchase'] = dataSearchAndSelling['search_term']==(dataSearchAndSelling['marca'])
dataSearchAndSelling["search_and_purchase"] = dataSearchAndSelling['search_and_purchase'].map(lambda x: 1 if x == True else 0)
dataSearchAndSelling = dataSearchAndSelling.drop(['search_term','condition','model','marca'],axis = 1)
dataSearchAndSelling = dataSearchAndSelling.groupby('person').agg({'Cantidad_De_Busquedas_Por_Termino':'sum', 'search_and_purchase':'sum'}).reset_index()


Feature que cuenta la cantidad de sku repetidos por usuario

In [126]:
#Aporto poco
skuData = pd.DataFrame({'cantidad_De_Compras_De_Mismo_Sku' :eventsData.groupby(['sku','event','person']).size()})
skuData = skuData.reset_index()
skuData = skuData[skuData['event'].map(lambda x: ( x== 'conversion'))]
skuData = skuData.drop(['event'],axis=1)
skuData = skuData.groupby('person').agg({'sku':'first', 'cantidad_De_Compras_De_Mismo_Sku':'sum' }).reset_index()

Inicializacion de dataframes que se usaran despues

In [127]:
geoData = eventsData[["person","event","city","region","country"]]

In [128]:
deviceUsersData = eventsData[["person","browser_version","operating_system_version","screen_resolution","device_type"]]

In [129]:
productData = eventsData[["person","event","sku","model","condition","storage","skus"]]

In [130]:
infoUserData = eventsData[["person","timestamp","search_term","staticpage","campaign_source","search_engine","channel","new_vs_returning"]]

In [131]:
timeData = eventsData[["person","timestamp","Parte del dia","event"]]

In [132]:
personColor = eventsData[["person","event","color"]]

Busco crear nuevo features 

temporales

In [133]:
users_partedeldia = timeData[["person","Parte del dia"]]

In [134]:
users_partedeldia = pd.get_dummies(users_partedeldia, columns=["Parte del dia"])

In [135]:
users_partedeldia = users_partedeldia.groupby("person",as_index = False).sum()

In [136]:
#Funcion que pasa a formato datetime las fechas que recibimos.
def datetime(date,format_string):
    return pd.to_datetime(date,format= format_string,errors='coerce')

timeData["timestamp"]= datetime(timeData["timestamp"],"%Y-%m-%dT%H:%M:%S")

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Mapeado de las fechas

In [137]:
timeData["year"] = timeData["timestamp"].map(lambda x: x.year)
timeData["month"] = timeData["timestamp"].map(lambda x: x.month)
timeData["day"] = timeData["timestamp"].map(lambda x: x.day)
timeData["hour"] = timeData["timestamp"].map(lambda x: x.hour)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

Feature que analiza los ultimos 15 dias

In [138]:
last_15days = timeData[["person","event","month","day"]]

In [139]:
last_15days = last_15days[last_15days["month"] == 5]

In [140]:
last_15days = last_15days[last_15days["day"] > 14]
last_15days = last_15days[["person","event"]]


In [141]:
last_15days["count_activity"] = 1

In [142]:
last_15days = pd.get_dummies(last_15days, columns=["event"])

In [143]:
last_15days = last_15days.groupby("person",as_index=False).sum()
last_15days = last_15days[["person","count_activity","event_conversion","event_checkout","event_viewed product"]]


In [144]:
last_15days.columns = ["person","count_activity","conversiones_15lastdays","checkout_15lastdays","viewed_15lastday"]

Nuevo formateo de timedata

In [145]:
timeData["minute"] = timeData["timestamp"].map(lambda x: x.minute)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Se elimina el year por ser todos 2018

In [146]:
timeData["year"].value_counts()

2018    2341681
Name: year, dtype: int64

In [147]:
timeData = timeData.drop("year",axis = 1)

Nuevo formateo de TimeData

In [148]:
timeData = pd.get_dummies(timeData, columns=["event"])

In [149]:
timeData = timeData.drop(["timestamp","Parte del dia"], axis=1)

In [150]:
timeData = timeData.drop("minute",axis=1)

Feature con sesiones de los usuarios

In [151]:
secionesUsuarios = timeData.groupby(["person","month","day","hour"]).sum()

In [152]:
secionesUsuarios["cant_seciones"] = 1

In [153]:
secionesUsuarios = secionesUsuarios[["cant_seciones","event_checkout","event_conversion","event_ad campaign hit","event_lead"]]


In [154]:
secionesUsuarios = secionesUsuarios.reset_index()

In [155]:
secionesUsuarios = secionesUsuarios.drop(["day","hour","month"],axis=1)

In [156]:
secionesUsuarios = secionesUsuarios.groupby(["person"]).sum().fillna(0)

In [157]:
secionesUsuarios = secionesUsuarios.reset_index()

Por dispositivo que usa el usuario para entrar al sitio

In [158]:
deviceUsersData["operating_system_version"].map(lambda x: str(x).split(" ")[0]).value_counts()

Windows       1262240
Android        975740
iOS             73279
Mac             10206
Linux            9456
Ubuntu           3313
nan              2870
Chrome           2376
Other            1048
Tizen             531
BlackBerry        518
Fedora             92
FreeBSD             8
Symbian             4
Name: operating_system_version, dtype: int64

In [159]:
deviceUsersData["operating_system_version"] = deviceUsersData["operating_system_version"].map(lambda x: str(x).split(" ")[0])

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [160]:
deviceUsersData["device_type"] = deviceUsersData["device_type"].map(lambda x: "otros" if(x == "Tablet" or x=="Unknown") else x)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [161]:
device_user_type = deviceUsersData[["person","device_type"]]

In [162]:
device_user_type["count"] = 1

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [163]:
device_user_type = device_user_type.groupby(["person","device_type"]).max().unstack().fillna(0).reset_index()

In [164]:
multiindex = device_user_type.columns
multiindex = multiindex.tolist()
multiindex
ind = pd.Index([ (e[1] if e[0]!= "person" else e[0]) for e in multiindex])
device_user_type.columns = ind


In [165]:
device_user_system = deviceUsersData[["person","operating_system_version"]]

In [166]:
device_user_system["count"] = 1

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [167]:
device_user_system = device_user_system.groupby(["person","operating_system_version"]).max().unstack().fillna(0).reset_index()

In [168]:
multiindex = device_user_system.columns
multiindex = multiindex.tolist()
multiindex
ind = pd.Index([ (e[1] if e[0]!= "person" else e[0]) for e in multiindex])
device_user_system.columns = ind

In [169]:
device_user_system = device_user_system[["person","Windows","Android","iOS"]]

Informacion usuarios

In [170]:
dic_terminos = {}
for busqueda in infoUserData["search_term"]:
    for term in str(busqueda).split(" "):
        if (not term in dic_terminos.keys()):
            dic_terminos[term] = 1
        else:
            dic_terminos[term]+=1     

In [171]:
list_terminos = sorted(dic_terminos.items(), key=lambda kv: kv[1])

In [172]:
list_terminos = list_terminos[::-1]

In [173]:
list_terminos.pop(0)

('nan', 2227918)

In [174]:
terminos = []
for name,count in list_terminos:
    if(len(name)>2):
        terminos.append((name,count))    

In [175]:
terminos_populares = terminos[:300]

In [176]:
infoUserData["search_term"] = infoUserData["search_term"].fillna("0")

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [177]:
def verificarPopularidadElTermino(string,terminospopulares):
    if (string==0):
        return False
    for palabra in string.split(" "):
        
        if(palabra in [name for name,count in terminospopulares]):
            return True
        
    return False

In [178]:
infoUserData["Busca_terminos_populares"] = infoUserData["search_term"].map(lambda x:1 if(verificarPopularidadElTermino(x,terminos_populares)) else 0)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Feature con el campaign_source y channel mas populares

In [179]:
person_campaing = infoUserData[["person","campaign_source"]]
person_channel = infoUserData[["person","channel"]]

In [180]:
person_term = infoUserData[["person","Busca_terminos_populares"]]

In [181]:
person_term = person_term.groupby("person",as_index = False).sum()

In [182]:
person_campaing = person_campaing.dropna()

In [183]:
person_campaing = pd.get_dummies(person_campaing, columns=["campaign_source"])

In [184]:
person_campaing = person_campaing[["person","campaign_source_google","campaign_source_criteo","campaign_source_rtbhouse"]]

In [185]:
person_campaing = person_campaing.groupby("person",as_index=False).sum()

In [186]:
person_channel = person_channel.dropna()

In [187]:
person_channel = pd.get_dummies(person_channel, columns=["channel"])

In [188]:
person_channel = person_channel[["person","channel_Direct","channel_Paid","channel_Organic","channel_Referral"]]

In [189]:
person_channel = person_channel.groupby("person",as_index=False).sum()

Productos visto por el usuario

In [190]:
productData["model"] = productData["model"].fillna("Desconocido")
productData["marca"] = productData["model"].map(lambda x:"apple" if(x.split(" ")[0] in ["iphone","ipad"]) else x.split(" ")[0])

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [191]:
productData["version"] = productData["model"].map(lambda x:" ".join(x.split(" ")[0:2]) if(x.split(" ")[0] in ["iphone","ipad"]) else " ".join(x.split(" ")[1:3]))

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [192]:
users_view_list = productData[["person","skus"]]

In [193]:
productData = productData[(productData["event"] == "checkout")]

In [194]:
productData = productData[["person","marca","condition","version","storage"]]

In [195]:
productData= pd.get_dummies(productData, columns=["marca","condition","version","storage"])
productDataDummies = productData

In [196]:
productData = productData.groupby("person",as_index=False).sum()

In [197]:
productData = productData.drop(["marca_quantum","condition_Novo"],axis = 1)

In [198]:
productData = productData[['person', 'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom',"version_galaxy j7","version_galaxy j5","version_galaxy s7","version_iphone 6","version_iphone 5s","version_iphone 6s","version_moto g4","version_moto g2","version_galaxy s6",'storage_128GB', 'storage_16GB', 'storage_256GB', 'storage_32GB',
       'storage_4GB', 'storage_512MB', 'storage_64GB', 'storage_8GB']]

In [199]:
users_view_list["skus"] = users_view_list["skus"].fillna(0)

In [200]:
users_view_list["skus"] = users_view_list["skus"].map(lambda x:0 if(x == 0) else 1)

In [201]:
users_view_list = users_view_list.groupby("person",as_index=False).max()

Feature de new vs returning

In [202]:
new_last15days = infoUserData[["person","timestamp","new_vs_returning"]]

In [203]:
new_last15days["timestamp"]= datetime(new_last15days["timestamp"],"%Y-%m-%dT%H:%M:%S")
new_last15days["month"] = new_last15days["timestamp"].map(lambda x: x.month)
new_last15days["day"] = new_last15days["timestamp"].map(lambda x: x.day)



/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

In [204]:
new_last15days = new_last15days[new_last15days["month"] == 5]
new_last15days = new_last15days[new_last15days["day"] >14]

In [205]:
new_last15days["new_15lastdays"] = new_last15days["new_vs_returning"] == "New"

In [206]:
new_last15days["new_15lastdays"].value_counts()

False    1377983
True       25484
Name: new_15lastdays, dtype: int64

In [207]:
new_last15days = new_last15days[["person","new_15lastdays"]]


In [208]:
new_last15days = new_last15days.groupby("person",as_index=False).max()

In [209]:
new_last15days["new_15lastdays"] = new_last15days["new_15lastdays"].map(lambda x: 1 if(x) else 0) 

Features geograficos

In [210]:
geoData = geoData[geoData["event"] == "conversion"]

In [211]:
region = geoData[["person","region"]]

In [212]:
region = region.drop_duplicates(subset = "person", keep = "first")

In [213]:
(region["region"].value_counts() > 50).value_counts()

False    29
True     15
Name: region, dtype: int64

In [214]:
region["region"] = region["region"].fillna(0)

In [215]:
sell_region = region["region"].value_counts()

In [216]:
region["ventas_region"] = region["region"].map(lambda x: sell_region[x]) 

In [217]:
esBrasilero = geoData[["person","country"]]

In [218]:
esBrasilero["Brasilero"] = esBrasilero["country"].map(lambda x: 1 if(x == "Brazil") else 0)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [219]:
esBrasilero = esBrasilero[["person","Brasilero"]]

In [220]:
esBrasilero = esBrasilero.groupby("person",as_index=False).max()

Feature con Colores

In [221]:
personColor["color"].value_counts().head()

Preto             314925
Dourado           275212
Cinza espacial    151770
Branco            126496
Prateado          114284
Name: color, dtype: int64

In [222]:
personColor = personColor.dropna()

In [223]:
bestcolorseller = personColor[personColor["event"] == "checkout"]
colorsellers = bestcolorseller["color"].value_counts()

In [224]:
personColor = pd.get_dummies(personColor, columns=["color"])

In [225]:
personColor = personColor[personColor["event"] == "viewed product"]

In [226]:
personColor = personColor[["person","color_Preto","color_Dourado","color_Branco","color_Cinza espacial"]]


In [227]:
personColor = personColor.groupby("person").sum()

Feature con la probabilidad de compra

In [228]:
versiones = productDataDummies

#65315 son las cantidades de checkouts
versiones['marca_apple'] = (versiones['marca_apple'].sum())/65315
versiones['marca_asus'] = (versiones['marca_asus'].sum())/65315
versiones['marca_lenovo'] = (versiones['marca_lenovo'].sum())/65315
versiones['marca_lg']= (versiones['marca_lg'].sum())/65315
versiones['marca_motorola']= (versiones['marca_motorola'].sum())/65315
versiones['marca_quantum']= (versiones['marca_quantum'].sum())/65315
versiones['marca_samsung']= (versiones['marca_samsung'].sum())/65315
versiones['marca_sony']= (versiones['marca_sony'].sum())/65315


In [229]:
comprasPorMarcaData = eventsData[["person","event","model"]]
comprasPorMarcaData = comprasPorMarcaData[comprasPorMarcaData['event'].map(lambda x: x =='checkout')]
comprasPorMarcaData["model"] = comprasPorMarcaData["model"].fillna("Desconocido")
comprasPorMarcaData["marca"] = comprasPorMarcaData["model"].map(lambda x:"apple" if(x.split(" ")[0] in ["iphone","ipad"]) else x.split(" ")[0])
comprasPorMarcaData = pd.get_dummies(comprasPorMarcaData, columns = ['marca'])
comprasPorMarcaData['marca_apple'] = comprasPorMarcaData['marca_apple'].map(lambda x: x* 0.409875)
comprasPorMarcaData['marca_asus'] = comprasPorMarcaData['marca_asus'].map(lambda x: x* 0.001562)
comprasPorMarcaData['marca_lenovo'] = comprasPorMarcaData['marca_lenovo'].map(lambda x: x* 0.008773)
comprasPorMarcaData['marca_lg'] = comprasPorMarcaData['marca_lg'].map(lambda x: x* 0.017454)
comprasPorMarcaData['marca_motorola'] = comprasPorMarcaData['marca_motorola'].map(lambda x: x* 0.129251)
comprasPorMarcaData['marca_quantum'] = comprasPorMarcaData['marca_quantum'].map(lambda x: x* 0.000505)
comprasPorMarcaData['marca_samsung'] = comprasPorMarcaData['marca_samsung'].map(lambda x: x* 0.426211)
comprasPorMarcaData['marca_sony'] = comprasPorMarcaData['marca_sony'].map(lambda x: x* 0.006369)
comprasPorMarcaData = comprasPorMarcaData.groupby('person').agg({'marca_apple':'sum','marca_asus':'sum','marca_lenovo':'sum','marca_lg':'sum','marca_motorola':'sum','marca_quantum':'sum','marca_samsung':'sum','marca_sony':'sum'})
comprasPorMarcaData.columns = ['Prob_Apple','Prob_Asus','Prob_Lenovo','Prob_lg','Prob_motorola','Prob_quantum','Prob_samsung','Prob_sony']

Feature con el screen resolution

In [230]:
resolutionData = eventsData[['person','event','screen_resolution']]
resolutionData = resolutionData[resolutionData['event'].map(lambda x: x == 'conversion')]
resolutionData = resolutionData.fillna('')
resolutionData['screen_resolution'] = resolutionData['screen_resolution'].map(lambda x: 1 if( x == '360x640') else 0)
resolutionData = resolutionData.groupby('person').agg({'screen_resolution':'sum'})
resolutionData['screen_resolution'] = resolutionData['screen_resolution'].map(lambda x: 1 if (x>0) else 0)
resolutionData

,screen_resolution
person,
000ba417,0
001001be,1
001804a2,0
0019e639,0
001b0bf9,0
0020152e,0
002aea56,0
002ed810,0
0038a117,0


Mergeo todos mis resultados de los nuevos features

In [231]:
secionesUsuariosReducido = secionesUsuarios#[["person","cant_seciones5","cant_seciones4","cant_seciones3","event_checkout5","event_checkout4","event_conversion5","event_conversion4","event_ad campaign hit4","event_ad campaign hit5","event_lead5","event_lead4"]]

In [232]:
dataTrocafone = pd.merge(secionesUsuariosReducido,users_view_list,on="person",how = "outer")
dataTrocafone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 38828
Data columns (total 7 columns):
person                   38829 non-null object
cant_seciones            38829 non-null int64
event_checkout           38829 non-null float64
event_conversion         38829 non-null float64
event_ad campaign hit    38829 non-null float64
event_lead               38829 non-null float64
skus                     38829 non-null int64
dtypes: float64(4), int64(2), object(1)
memory usage: 2.4+ MB


In [234]:
dataTrocafone = pd.merge(dataTrocafone,resolutionData.reset_index(),on="person",how = "outer")
#dataTrocafone['screen_resolution_A'] = dataTrocafone['screen_resolution_A'].fillna(dataTrocafone['screen_resolution_A'].mean())
dataTrocafone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 38828
Data columns (total 8 columns):
person                   38829 non-null object
cant_seciones            38829 non-null int64
event_checkout           38829 non-null float64
event_conversion         38829 non-null float64
event_ad campaign hit    38829 non-null float64
event_lead               38829 non-null float64
skus                     38829 non-null int64
screen_resolution        4293 non-null float64
dtypes: float64(5), int64(2), object(1)
memory usage: 2.7+ MB


In [235]:
dataTrocafone = pd.merge(dataTrocafone,person_term,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [236]:
dataTrocafone = pd.merge(dataTrocafone,device_user_type,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [237]:
dataTrocafone["Computer"] = dataTrocafone["Computer"].fillna(0)
dataTrocafone['person'].count()

38829

In [238]:
dataTrocafone["Smartphone"] = dataTrocafone["Smartphone"].fillna(0)
dataTrocafone['person'].count()

38829

In [239]:
dataTrocafone["otros"] = dataTrocafone["otros"].fillna(0)
dataTrocafone['person'].count()

38829

In [240]:
dataTrocafone = pd.merge(dataTrocafone,productData,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [241]:
dataTrocafone = pd.merge(dataTrocafone,users_partedeldia,on="person",how="left")
dataTrocafone['person'].count()

38829

In [242]:
dataTrocafone = pd.merge(dataTrocafone,person_campaing,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [243]:
dataTrocafone = pd.merge(dataTrocafone,person_channel,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [244]:
dataTrocafone = pd.merge(dataTrocafone,esBrasilero,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [245]:
dataTrocafone = dataTrocafone.fillna(0)

In [246]:
dataTrocafone = pd.merge(dataTrocafone,region,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [247]:
dataTrocafone = dataTrocafone.drop("region",axis=1)

In [248]:
dataTrocafone["ventas_region"] = dataTrocafone["ventas_region"].fillna(0)

In [249]:
dataTrocafone = pd.merge(dataTrocafone,last_15days,how = "left", on="person")
dataTrocafone['person'].count()

38829

In [250]:
dataTrocafone["count_activity"] = dataTrocafone["count_activity"].fillna(0)

In [251]:
dataTrocafone["conversiones_15lastdays"] = dataTrocafone["conversiones_15lastdays"].fillna(0)
dataTrocafone["checkout_15lastdays"] = dataTrocafone["checkout_15lastdays"].fillna(0)
dataTrocafone["viewed_15lastday"] = dataTrocafone["viewed_15lastday"].fillna(0)

In [252]:
personColor = personColor.reset_index()

In [253]:
dataTrocafone = pd.merge(dataTrocafone,personColor,how = "left", on="person")
dataTrocafone['person'].count()

38829

In [254]:
dataTrocafone["color_Preto"] = dataTrocafone["color_Preto"].fillna(0)
dataTrocafone["color_Dourado"] = dataTrocafone["color_Dourado"].fillna(0)
dataTrocafone["color_Branco"] = dataTrocafone["color_Branco"].fillna(0)
dataTrocafone["color_Cinza espacial"] = dataTrocafone["color_Cinza espacial"].fillna(0)

In [255]:
dataTrocafone = pd.merge(dataTrocafone,new_last15days,how = "left", on="person")
dataTrocafone['person'].count()

38829

In [256]:
dataTrocafone["new_15lastdays"] = dataTrocafone["new_15lastdays"].fillna(0)


In [257]:
dataTrocafone = pd.merge(dataTrocafone,skuData,how = "left", on="person")
dataTrocafone = pd.merge(dataTrocafone,dataSearchAndSelling,how = "left", on="person")
dataTrocafone['person'].count()

38829

In [258]:
dataTrocafone["cantidad_De_Compras_De_Mismo_Sku"] = dataTrocafone["cantidad_De_Compras_De_Mismo_Sku"].fillna(0)
dataTrocafone["sku"] = dataTrocafone["sku"].fillna(0)
dataTrocafone['person'].count()

38829

In [259]:
dataTrocafone["search_and_purchase"] = dataTrocafone["search_and_purchase"].fillna(0)
dataTrocafone["Cantidad_De_Busquedas_Por_Termino"] = dataTrocafone["Cantidad_De_Busquedas_Por_Termino"].fillna(0)
dataTrocafone['person'].count()

38829

In [260]:
dataTrocafone = pd.merge(dataTrocafone,auxiliar,how = "left", on="person")
dataTrocafone["cantidad_de_Url_de_compra"] = dataTrocafone["cantidad_de_Url_de_compra"].fillna(0)
dataTrocafone['person'].count()

38829

In [261]:
dataTrocafone = pd.merge(dataTrocafone,staticPageData,how = "left", on="person")
dataTrocafone["cantidadDeStaticPage"] = dataTrocafone["cantidadDeStaticPage"].fillna(0)
dataTrocafone['person'].count()

38829

In [263]:
dataTrocafone = pd.merge(dataTrocafone,comprasPorMarcaData.reset_index(),how = "left", on="person")
dataTrocafone = dataTrocafone.fillna(0)
dataTrocafone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 38828
Data columns (total 76 columns):
person                               38829 non-null object
cant_seciones                        38829 non-null int64
event_checkout                       38829 non-null float64
event_conversion                     38829 non-null float64
event_ad campaign hit                38829 non-null float64
event_lead                           38829 non-null float64
skus                                 38829 non-null int64
screen_resolution                    38829 non-null float64
Busca_terminos_populares             38829 non-null int64
Computer                             38829 non-null float64
Smartphone                           38829 non-null float64
otros                                38829 non-null float64
marca_apple                          38829 non-null float64
marca_asus                           38829 non-null float64
marca_lenovo                         38829 non-null float6

In [264]:
dataTrocafone = pd.merge(dataTrocafone,dataF,how = "left", on="person")
dataTrocafone["sku_multiple"] = dataTrocafone["sku_multiple"].fillna(0)

Armo el set de training y el de test

In [265]:
trainingSet = pd.read_csv("data/labels_training_set.csv", low_memory = False)

In [266]:
trainingSet["person"].count()

19414

In [267]:
dataTrocafone["person"].count()

38829

In [268]:
setTraining = pd.merge(dataTrocafone,trainingSet,on="person",how = "inner")

In [269]:
setTraining["person"].count()

19414

In [271]:
setTraining.to_csv('setEntrenamiento.csv',index=False)

In [272]:
kaggle_test = pd.read_csv("data/trocafone_kaggle_test.csv", low_memory = False)

In [273]:
kaggle_test["person"].count()

19415

In [274]:
test = pd.merge(kaggle_test,dataTrocafone,on="person",how="inner")

In [275]:
test["person"].count()

19415

In [276]:
test.to_csv("setTest.csv",index = False)

In [277]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19415 entries, 0 to 19414
Data columns (total 77 columns):
person                               19415 non-null object
cant_seciones                        19415 non-null int64
event_checkout                       19415 non-null float64
event_conversion                     19415 non-null float64
event_ad campaign hit                19415 non-null float64
event_lead                           19415 non-null float64
skus                                 19415 non-null int64
screen_resolution                    19415 non-null float64
Busca_terminos_populares             19415 non-null int64
Computer                             19415 non-null float64
Smartphone                           19415 non-null float64
otros                                19415 non-null float64
marca_apple                          19415 non-null float64
marca_asus                           19415 non-null float64
marca_lenovo                         19415 non-null float6